# Santander Dev Week 2023 (ETL com Python)

#### Como criar sua chave para API da Steam

Para obter uma chave de API da Steam e começar a usar a Steam Web API, siga estes passos:

1. **Crie uma Conta de Desenvolvedor na Steam**:  
Se você ainda não possui uma conta de desenvolvedor na Steam, crie uma em Steamworks. Você precisará de uma conta Steam válida para fazer isso.

2. **Configure um Novo Aplicativo**:  
Após criar sua conta de desenvolvedor, faça login no Steamworks e vá para a seção "Meus aplicativos". Clique em "Registrar um aplicativo".

3. **Preencha as Informações do Aplicativo**:
    - Preencha os detalhes do seu aplicativo, incluindo nome, descrição e categoria.
    - Selecione o tipo de API que deseja usar (geralmente "Web API").
    - Escolha a permissão adequada para o seu aplicativo, dependendo das informações que você planeja acessar. Por exemplo, se você deseja acessar a lista de jogos em uma biblioteca de um usuário, você precisará da permissão "Usuário: Biblioteca de jogos".

4. **Receba sua Chave de Desenvolvedor**:  
Após registrar o aplicativo, você receberá uma "Chave de publicação" (publish key) e uma "Chave secreta" (secret key). A "Chave secreta" é usada para autenticar as chamadas da API e não deve ser compartilhada publicamente. Mantenha essa chave em segurança.

5. **Use sua Chave de API em seu Aplicativo**:  
Você pode começar a usar sua chave de API em seu aplicativo para fazer chamadas à Steam Web API. Certifique-se de que o aplicativo solicite as permissões corretas aos usuários e siga as políticas e restrições da Steam para o uso da API.

6. **Limite de Taxa de Solicitação**:  
Lembre-se de que a Steam Web API impõe limites de taxa de solicitação. Certifique-se de não exceder esses limites para evitar bloqueios de acesso à API.

7. **Respeite os Termos de Serviço da Steam**:  
Certifique-se de que seu uso da Steam Web API esteja em conformidade com os Termos de Serviço da Steam e as políticas de desenvolvimento da Valve. A Valve pode revogar seu acesso à API se você violar esses termos.

#### Como encontrar o seu SteamID

https://help.steampowered.com/pt-br/faqs/view/2816-BE67-5B69-0FEC


## Configurando as Keys

In [43]:
import requests
import json
import credentials

# Crie um arquivo credentials.py contend:
# api_key = 'YOUR_API_KEY'
# steam_id = 'YOUR_STEAM_ID'
# Substitua:
#   'YOUR_API_KEY' pela sua chave de API de desenvolvedor Steam
#   'YOUR_STEAM_ID' pelo seu ID Steam (ou do amigo que deseja consultar)

api_key = credentials.api_key

steam_id = credentials.steam_id


## Extract

In [44]:
# @title
# recupera a lista de jogos que possui:
def get_game_list():
  url = f'https://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={api_key}&steamid={steam_id}&format=json&include_appinfo=true'
  try:
    response = requests.get(url)
    return response.json() if response.status_code == 200 else None

  except Exception as e:
    return {
        'success': False,
        'message': f'Erro: {str(e)}'
    }

my_games = get_game_list()
# print(json.dumps(my_games, indent=2))

In [ ]:
# Recuperar a lista de amigos na Steam

def friend_list():
  # URL da Steam Web API para recuperar a lista de amigos
  url = f'http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={api_key}&steamid={steam_id}&relationship=friend&format=json'
  try:
    response = requests.get(url)
    return response.json() if response.status_code == 200 else None

  except Exception as e:
    return None

my_friends = friend_list()['friendslist']['friends']
# print(json.dumps(my_friends, indent=2))

In [ ]:
# Recupera os dados dos 100 primeiros amigo da lista - Restrição da Steam

def get_friends_details(friends):

  friend_ids = ''
  for index, friend in enumerate(friends):
      if index > 100:
          break
      friend_ids += (',' if len(friend_ids) > 0 else '') + friend['steamid']

  url = f'http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v2/?key={api_key}&steamids={friend_ids}'

  friends_details = requests.get(url)

  return friends_details.json() if friends_details.status_code == 200 else None

my_friends_detailed_list = get_friends_details(my_friends)
# print(json.dumps(my_friends_detailed_list, indent=2))

In [ ]:
# Recupera os dados de um amigo

def get_friend_detail(friend):

  friend_id = friend['steamid']

  url = f'http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v2/?key={api_key}&steamids={friend_id}'
  friend_detail = requests.get(url)

  if friend_detail.status_code == 200:
    friend_detail = friend_detail.json()
    friend_detail = friend_detail['response']['players'][0]
    return friend_detail
  else:
    return None

friend_detail = get_friend_detail(my_friends[0])
# print(json.dumps(friend_detail, indent=2))

In [ ]:
# Recupera a lista de jogos de um amigo
def friend_games_list(friend):
  friend_id = friend['steamid']
  games_url = f'http://api.steampowered.com/IPlayerService/GetOwnedGames/v1/?key={api_key}&steamid={friend_id}&format=json&include_appinfo=true&appids_filter'
  games_response = requests.get(games_url)

  return games_response.json() if games_response.status_code == 200 else None

games_list = friend_games_list(my_friends[1])
# print(json.dumps(games_list, indent=2))

## Transform


In [ ]:
# Adiciona aos Amigos seus respectivos jogos
for index, friend in enumerate(my_friends_detailed_list['response']['players']):
  friend_games = friend_games_list(friend)
  friend.setdefault("games", friend_games)



## Load

In [ ]:
# Lista dos amigos com seus respectivos jogos
print(json.dumps(my_friends_detailed_list, indent=2))

# Lista dos seus jogos
print(json.dumps(my_games, indent=2))
